<a href="https://colab.research.google.com/github/cheonghf/ML-P4-03/blob/main/V2_Project_SourceCode_P4_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load CSV, Classify Data and Re-arrange Data

In [137]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset ID
dataset_id = "d_3c55210de27fcccda2ed0c63fdd2b352"

# API URL for the dataset
dataset_url = f"https://data.gov.sg/api/action/datastore_search?resource_id={dataset_id}&limit=5000"

# Request data from the API
response = requests.get(dataset_url)
response.raise_for_status()  # Ensure the request was successful

# Convert response to JSON
data = response.json()

# Extract records
if 'result' in data and 'records' in data['result']:
    df_ges1 = pd.DataFrame(data['result']['records'])
    df_ges1 = df_ges1.drop(columns=['_id'], errors='ignore')
    df_ges2 = df_ges1.copy() #df_ges2 will be the DataFrame we will be manipulating
    print("DataFrame ges 1 (R , C):", df_ges1.shape)  # Print shape to verify
    print("DataFrame ges 2 (R , C):", df_ges2.shape)  # Print shape to verify
else:
    print("No records found in the dataset.")

# dataset_1 = 'https://raw.githubusercontent.com/cheonghf/ML-P4-03/refs/heads/main/GraduateEmploymentSurveyNTUNUSSITSMUSUSSSUTD.csv'
# df_ges1 = pd.read_csv(dataset_1)
# https://data.gov.sg/datasets/d_3c55210de27fcccda2ed0c63fdd2b352/view?dataExplorerPage=2

DataFrame ges 1 (R , C): (1262, 12)
DataFrame ges 2 (R , C): (1262, 12)


In [138]:
# Define clusters based on degree names
def classify_degree(degree):
    engineering_keywords = ['Engineering', 'Material Science', 'Mechanical', 'Electrical', 'Civil', 'Aerospace', 'Bioengineering']
    it_keywords = ['Computing', 'Computer Science', 'Information Systems', 'Software', 'Cybersecurity', 'Data Science']
    science_keywords = ['Science', 'Physics', 'Mathematics', 'Chemistry', 'Biological', 'Pharmacy', 'Life Sciences']
    business_keywords = ['Business', 'Finance', 'Accountancy', 'Economics', 'Management', 'Marketing']
    arts_keywords = ['Arts', 'Design', 'Music', 'Theatre', 'Performing', 'Linguistics', 'Philosophy', 'History', 'Literature']

    degree_lower = degree.lower()

    if any(keyword.lower() in degree_lower for keyword in engineering_keywords):
        return 'Engineering'
    elif any(keyword.lower() in degree_lower for keyword in it_keywords):
        return 'Information Technology'
    elif any(keyword.lower() in degree_lower for keyword in science_keywords):
        return 'Science'
    elif any(keyword.lower() in degree_lower for keyword in business_keywords):
        return 'Business'
    elif any(keyword.lower() in degree_lower for keyword in arts_keywords):
        return 'Arts'
    else:
        return 'Others'

# Apply classification to the dataset
df_ges2['degree_cluster'] = df_ges2['degree'].apply(classify_degree)
print("Updated DataFrame ges 2 (R , C):",df_ges2.shape)
print("")
print("DataType of DataFrame ges 2:")
print(df_ges2.dtypes)

Updated DataFrame ges 2 (R , C): (1262, 13)

DataType of DataFrame ges 2:
year                         object
university                   object
school                       object
degree                       object
employment_rate_overall      object
employment_rate_ft_perm      object
basic_monthly_mean           object
basic_monthly_median         object
gross_monthly_mean           object
gross_monthly_median         object
gross_mthly_25_percentile    object
gross_mthly_75_percentile    object
degree_cluster               object
dtype: object


In [139]:
columns_to_convert = [
    "employment_rate_overall", "employment_rate_ft_perm",
    "basic_monthly_mean", "basic_monthly_median",
    "gross_monthly_mean", "gross_monthly_median",
    "gross_mthly_25_percentile", "gross_mthly_75_percentile"
]

for column in columns_to_convert:
    # Replace commas or other problematic characters if needed
    df_ges2[column] = df_ges2[column].astype(str).str.replace(',', '')

    # Remove leading/trailing whitespace
    df_ges2[column] = df_ges2[column].str.strip()

    # Convert to numeric
    df_ges2[column] = pd.to_numeric(df_ges1[column], errors='coerce')

# Check if conversion worked
print("Reconfirm DataFrame ges 2 (R , C):", df_ges2.shape)
print("")
print("Updated DataType of DataFrame ges 2:")
print(df_ges2.dtypes)

Reconfirm DataFrame ges 2 (R , C): (1262, 13)

Updated DataType of DataFrame ges 2:
year                          object
university                    object
school                        object
degree                        object
employment_rate_overall      float64
employment_rate_ft_perm      float64
basic_monthly_mean           float64
basic_monthly_median         float64
gross_monthly_mean           float64
gross_monthly_median         float64
gross_mthly_25_percentile    float64
gross_mthly_75_percentile    float64
degree_cluster                object
dtype: object


In [140]:
# Grouping by year, university, and degree classification while removing unnecessary columns
df_ges2 = df_ges2.drop(columns=['school', 'degree']).groupby(['year', 'degree_cluster', 'university', ]).mean().reset_index()

# Check if grouping worked
print("Updated DataFrame ges 2 (R , C):", df_ges2.shape)
print("")
print("Reconfirm DataType of DataFrame ges 2:")
print(df_ges2.dtypes)

Updated DataFrame ges 2 (R , C): (237, 11)

Reconfirm DataType of DataFrame ges 2:
year                          object
degree_cluster                object
university                    object
employment_rate_overall      float64
employment_rate_ft_perm      float64
basic_monthly_mean           float64
basic_monthly_median         float64
gross_monthly_mean           float64
gross_monthly_median         float64
gross_mthly_25_percentile    float64
gross_mthly_75_percentile    float64
dtype: object


#Re-Check for Null Values in DataFrame, Replace str 'Na' with Null and Remove Null Values

In [141]:
# Copy ges1 DataFrame into another DataFrame
df_ges2_clean = df_ges2[:]

print("DataFrame ges 2 (R , C):", df_ges2.shape)  # Print shape to verify
print("DataFrame ges 2 CLEAN (R , C):", df_ges2_clean.shape)  # Print shape to verify
print("")
print("Check for Null: BEFORE Replacing Potential na strings with Null values")
df_ges2_clean.isnull().sum(axis=0)

DataFrame ges 2 (R , C): (237, 11)
DataFrame ges 2 CLEAN (R , C): (237, 11)

Check for Null: BEFORE Replacing Potential na strings with Null values


,0
year,0
degree_cluster,0
university,0
employment_rate_overall,3
employment_rate_ft_perm,3
basic_monthly_mean,3
basic_monthly_median,3
gross_monthly_mean,3
gross_monthly_median,3
gross_mthly_25_percentile,3


In [142]:
# Replace the string 'na' with actual NaN (np.nan)
df_ges2_clean.replace('na', np.nan, inplace=True)

# Count NaN values in each column
na_counts = df_ges2_clean.isna().sum()

print("DataFrame ges 2 CLEAN (R , C):", df_ges2_clean.shape)  # Print shape to verify
print("")
print("Check for Null: AFTER Replacing Potential na strings with Null values")
print(na_counts)

DataFrame ges 2 CLEAN (R , C): (237, 11)

Check for Null: AFTER Replacing Potential na strings with Null values
year                         0
degree_cluster               0
university                   0
employment_rate_overall      3
employment_rate_ft_perm      3
basic_monthly_mean           3
basic_monthly_median         3
gross_monthly_mean           3
gross_monthly_median         3
gross_mthly_25_percentile    3
gross_mthly_75_percentile    3
dtype: int64


In [143]:
# Remove rows with any NaN values
df_ges2_clean = df_ges2_clean.dropna()

# Check if Null Cleaning worked
print("Final Cleaned DataFrame ges 2 (R , C):", df_ges2_clean.shape)
print("")
print("Final Cleaned DataType of DataFrame ges 2:")
print(df_ges2_clean.isnull().sum(axis=0))

Final Cleaned DataFrame ges 2 (R , C): (234, 11)

Final Cleaned DataType of DataFrame ges 2:
year                         0
degree_cluster               0
university                   0
employment_rate_overall      0
employment_rate_ft_perm      0
basic_monthly_mean           0
basic_monthly_median         0
gross_monthly_mean           0
gross_monthly_median         0
gross_mthly_25_percentile    0
gross_mthly_75_percentile    0
dtype: int64


#Making Sense of the Data

In [144]:
df_ges2_clean.to_csv("my_data.csv", index=False)

#AFTER

In [21]:
#Extract NTU and NUS DataFrames respectively
df_clean_ges_ntu = df_clean_ges.loc[df_clean_ges['university'] == 'Nanyang Technological University']
df_clean_ges_nus = df_clean_ges.loc[df_clean_ges['university'] == 'National University of Singapore']

# Group by year and school cluster (college level)
df_ntu_grouped = df_clean_ges_ntu.groupby(['year', 'university', 'school']).mean(numeric_only=True).reset_index()
df_nus_grouped = df_clean_ges_nus.groupby(['year', 'university', 'school']).mean(numeric_only=True).reset_index()

In [19]:
df_ntu_grouped.school.value_counts()

,count
school,
College of Business (Nanyang Business School),10
College of Engineering,10
National Institute of Education (NIE),10
"College of Humanities, Arts & Social Sciences",9
College of Sciences,7
Sports Science and Management,6
Lee Kong Chian School of Medicine,5
College of Science,3


In [20]:
df_nus_grouped.school.value_counts()

,count
school,
Faculty of Arts & Social Sciences,10
Faculty of Law,10
Faculty of Science,10
NUS Business School,10
School of Computing,10
School of Design & Environment,9
Faculty of Dentistry,8
Faculty of Engineering,7
Yale-NUS College,6


In [7]:
df_nus_grouped.dtypes

,0
year,int64
university,object
school,object
employment_rate_overall,float64
employment_rate_ft_perm,float64
basic_monthly_mean,float64
basic_monthly_median,float64
gross_monthly_mean,float64
gross_monthly_median,float64
gross_mthly_25_percentile,float64
